<a href="https://colab.research.google.com/github/pavanpyla/Project/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder , LabelEncoder
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
Train=pd.read_csv("/content/drive/MyDrive/fraud detection/Train-1542865627584.csv")
Train_Beneficiarydata=pd.read_csv("/content/drive/MyDrive/fraud detection/Train_Beneficiarydata-1542865627584.csv")
Train_Inpatientdata=pd.read_csv("/content/drive/MyDrive/fraud detection/Train_Inpatientdata-1542865627584.csv")
Train_Outpatientdata=pd.read_csv("/content/drive/MyDrive/fraud detection/Train_Outpatientdata-1542865627584.csv")

In [ ]:
Train_Allpatientdata=pd.merge(Train_Outpatientdata,Train_Inpatientdata,
                              left_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
                              right_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode']
                              ,how='outer')

In [ ]:
Train_Allpatientdata=pd.merge(Train_Allpatientdata,Train_Beneficiarydata,on="BeneID")

In [ ]:
df=pd.merge(Train_Allpatientdata,Train,on="Provider")

In [ ]:
df

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,2,2,2,2,2,0,0,30,50,Yes
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,1,1,1,1,2,0,0,1810,760,Yes
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,1,1,1,1,2,0,0,1810,760,Yes
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,1,1,1,1,2,0,0,1810,760,Yes
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,...,1,1,1,1,2,0,0,1810,760,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE57978,CLM37910,2009-02-11,2009-02-16,PRV53511,0,PHY378514,PHY426843,NaN,40491,...,1,1,2,1,2,2200,2136,30,100,No
558207,BENE80245,CLM79653,2009-12-15,2009-12-18,PRV51711,4000,PHY345109,NaN,NaN,2948,...,1,2,2,2,2,4000,1068,0,0,No
558208,BENE85177,CLM77745,2009-11-29,2009-12-07,PRV54503,8000,PHY418343,NaN,NaN,515,...,2,1,2,2,2,8000,1068,0,0,No
558209,BENE105577,CLM53155,2009-05-28,2009-05-31,PRV51239,5000,PHY382916,NaN,NaN,27651,...,1,1,2,2,2,5000,1068,0,0,No


In [ ]:
df=df.drop(['BeneID','ClaimID'],axis=1)

In [ ]:
df

,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,...,2,2,2,2,2,0,0,30,50,Yes
1,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,...,1,1,1,1,2,0,0,1810,760,Yes
2,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,...,1,1,1,1,2,0,0,1810,760,Yes
3,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,NaN,NaN,...,1,1,1,1,2,0,0,1810,760,Yes
4,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,4280,2449,...,1,1,1,1,2,0,0,1810,760,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,2009-02-11,2009-02-16,PRV53511,0,PHY378514,PHY426843,NaN,40491,41401,5859,...,1,1,2,1,2,2200,2136,30,100,No
558207,2009-12-15,2009-12-18,PRV51711,4000,PHY345109,NaN,NaN,2948,3129,25000,...,1,2,2,2,2,4000,1068,0,0,No
558208,2009-11-29,2009-12-07,PRV54503,8000,PHY418343,NaN,NaN,515,5119,4240,...,2,1,2,2,2,8000,1068,0,0,No
558209,2009-05-28,2009-05-31,PRV51239,5000,PHY382916,NaN,NaN,27651,25000,5715,...,1,1,2,2,2,5000,1068,0,0,No


In [ ]:
df.isna().sum()  

ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
ClmDiagnosisCode_3                 315156
ClmDiagnosisCode_4                 393675
ClmDiagnosisCode_5                 446287
ClmDiagnosisCode_6                 473819
ClmDiagnosisCode_7                 492034
ClmDiagnosisCode_8                 504767
ClmDiagnosisCode_9                 516396
ClmDiagnosisCode_10                553201
ClmProcedureCode_1                 534901
ClmProcedureCode_2                 552721
ClmProcedureCode_3                 557242
ClmProcedureCode_4                 558093
ClmProcedureCode_5                 558202
ClmProcedureCode_6                 558211
DeductibleAmtPaid                 

In [ ]:
df=df.drop(['ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7',
                'ClmDiagnosisCode_8','ClmDiagnosisCode_9','ClmDiagnosisCode_10',
                'ClmProcedureCode_1','ClmProcedureCode_2','ClmProcedureCode_3','ClmProcedureCode_4',
                'ClmProcedureCode_5','ClmProcedureCode_6',],axis=1)

In [ ]:
df

,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,DeductibleAmtPaid,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,0.0,...,2,2,2,2,2,0,0,30,50,Yes
1,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,0.0,...,1,1,1,1,2,0,0,1810,760,Yes
2,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,0.0,...,1,1,1,1,2,0,0,1810,760,Yes
3,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,NaN,0.0,...,1,1,1,1,2,0,0,1810,760,Yes
4,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,4280,0.0,...,1,1,1,1,2,0,0,1810,760,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,2009-02-11,2009-02-16,PRV53511,0,PHY378514,PHY426843,NaN,40491,41401,1068.0,...,1,1,2,1,2,2200,2136,30,100,No
558207,2009-12-15,2009-12-18,PRV51711,4000,PHY345109,NaN,NaN,2948,3129,1068.0,...,1,2,2,2,2,4000,1068,0,0,No
558208,2009-11-29,2009-12-07,PRV54503,8000,PHY418343,NaN,NaN,515,5119,1068.0,...,2,1,2,2,2,8000,1068,0,0,No
558209,2009-05-28,2009-05-31,PRV51239,5000,PHY382916,NaN,NaN,27651,25000,1068.0,...,1,1,2,2,2,5000,1068,0,0,No


In [ ]:
df.isna().sum()  

ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
DeductibleAmtPaid                     899
ClmAdmitDiagnosisCode              412312
AdmissionDt                        517737
DischargeDt                        517737
DiagnosisGroupCode                 517737
DOB                                     0
DOD                                554080
Gender                                  0
Race                                    0
RenalDiseaseIndicator                   0
State                                   0
County                                  0
NoOfMonths_PartACov                     0
NoOfMonths_PartBCov                     0
ChronicCond_Alzheimer             

In [ ]:
data=df

In [ ]:
df = pd.DataFrame()
data.isna().sum()

ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
DeductibleAmtPaid                     899
ClmAdmitDiagnosisCode              412312
AdmissionDt                        517737
DischargeDt                        517737
DiagnosisGroupCode                 517737
DOB                                     0
DOD                                554080
Gender                                  0
Race                                    0
RenalDiseaseIndicator                   0
State                                   0
County                                  0
NoOfMonths_PartACov                     0
NoOfMonths_PartBCov                     0
ChronicCond_Alzheimer             

In [ ]:

startdate= pd.to_datetime( data["AdmissionDt"] )
enddate= pd.to_datetime( data["DischargeDt"] )

pe = ( enddate - startdate).dt.days
df["Hperiod"] = pe

In [ ]:
df['claim']=data['InscClaimAmtReimbursed']

In [ ]:
startdate= pd.to_datetime( data["ClaimStartDt"] )
enddate= pd.to_datetime( data["ClaimEndDt"] )

p = ( enddate - startdate).dt.days
df["period"] = p

In [ ]:
def physician_same(row):
    atten_oper=row["AttendingPhysician"]==row["OperatingPhysician"]
    oper_other=row["OperatingPhysician"]==row["OtherPhysician"]
    atten_other=row["AttendingPhysician"]==row["OtherPhysician"]
    if atten_oper==True and oper_other==True:# atten = oper = other
        return 0
    elif atten_oper==True and oper_other==False:# atten = oper != other
        return 1
    elif atten_oper==False and oper_other==True:# atten != oper = other
        return 2
    else:# atten != oper != other
        return 3
    
phy_same=data.apply(physician_same,axis=1)

In [ ]:
df['phy_same']=phy_same

In [ ]:
from sklearn import preprocessing
  

label_encoder = preprocessing.LabelEncoder()
  

data['PotentialFraud']= label_encoder.fit_transform(data['PotentialFraud'])
  
data['PotentialFraud'].unique()

array([1, 0])

In [ ]:
data['PotentialFraud'].sum()

212796

In [ ]:
data['Gender']

0         2
1         1
2         1
3         1
4         1
         ..
558206    2
558207    1
558208    1
558209    2
558210    2
Name: Gender, Length: 558211, dtype: int64

In [ ]:
df["Gender"]=data["Gender"].replace({2:0})

In [ ]:
df

,Hperiod,claim,period,phy_same,Gender
0,NaN,30,0,3,0
1,NaN,40,0,3,1
2,NaN,200,0,3,1
3,NaN,20,0,3,1
4,NaN,40,0,3,1
...,...,...,...,...,...
558206,5.0,0,5,3,0
558207,3.0,4000,3,3,1
558208,8.0,8000,8,3,1
558209,3.0,5000,3,3,0


In [ ]:
df["RenalDiseaseIndicator"]=data["RenalDiseaseIndicator"].replace({"Y":1})

In [ ]:
birthdate=pd.to_datetime(data["DOB"])
enddate=pd.to_datetime(data["DOD"])

# cheack whether the patient dead or alive
def alife_function(value):
    if value==True:
        return 1
    else:
        return 0
alive = pd.isna(enddate).apply(alife_function)


# get the age of patient
max_date=enddate.dropna().max()
enddate[pd.isna(enddate)]=max_date
period=(((enddate-birthdate).dt.days/356).astype(int))

df["age"]=period
df["alife"]=alive

In [ ]:
data.isna().sum()

ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
DeductibleAmtPaid                     899
ClmAdmitDiagnosisCode              412312
AdmissionDt                        517737
DischargeDt                        517737
DiagnosisGroupCode                 517737
DOB                                     0
DOD                                554080
Gender                                  0
Race                                    0
RenalDiseaseIndicator                   0
State                                   0
County                                  0
NoOfMonths_PartACov                     0
NoOfMonths_PartBCov                     0
ChronicCond_Alzheimer             

In [ ]:
from pandas.api.types import is_numeric_dtype

In [ ]:
for i in df.columns:
  print(f"{i }:",is_numeric_dtype(df
[i]))

Hperiod: True
claim: True
period: True
phy_same: True
Gender: True
RenalDiseaseIndicator: False
age: True
alife: True


In [ ]:
columns=['NoOfMonths_PartACov','NoOfMonths_PartBCov','ChronicCond_Alzheimer','ChronicCond_Alzheimer','ChronicCond_KidneyDisease','ChronicCond_Cancer',
         'ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart',
         'ChronicCond_stroke','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt','PotentialFraud']

In [ ]:
df['Provider']= label_encoder.fit_transform(data['Provider'])
  
df['Provider'].unique()

array([4004, 5280, 5269, ..., 2810,  185, 2016])

In [ ]:
for i in columns:
  df[i]=data[i]

In [ ]:
df['Provider']=df['Provider']

In [ ]:
df

,Hperiod,claim,period,phy_same,Gender,RenalDiseaseIndicator,age,alife,Provider,NoOfMonths_PartACov,...,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_stroke,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,NaN,30,0,3,0,0,75,1,4004,12,...,2,2,2,2,2,2,2,30,50,1
1,NaN,40,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
2,NaN,200,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
3,NaN,20,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
4,NaN,40,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,5.0,0,5,3,0,0,68,0,2013,12,...,1,2,1,2,1,1,2,30,100,0
558207,3.0,4000,3,3,1,1,68,1,567,12,...,2,2,2,2,1,2,2,0,0,0
558208,8.0,8000,8,3,1,0,74,1,2810,12,...,2,2,2,2,2,1,2,0,0,0
558209,3.0,5000,3,3,0,0,84,1,185,12,...,2,2,2,2,1,1,2,0,0,0


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
from tensorflow import keras

,Hperiod,claim,period,phy_same,Gender,RenalDiseaseIndicator,age,alife,Provider,NoOfMonths_PartACov,...,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_stroke,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,NaN,30,0,3,0,0,75,1,4004,12,...,2,2,2,2,2,2,2,30,50,1
1,NaN,40,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
2,NaN,200,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
3,NaN,20,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
4,NaN,40,0,3,1,0,89,1,4004,12,...,2,2,2,2,1,1,2,1810,760,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,5.0,0,5,3,0,0,68,0,2013,12,...,1,2,1,2,1,1,2,30,100,0
558207,3.0,4000,3,3,1,1,68,1,567,12,...,2,2,2,2,1,2,2,0,0,0
558208,8.0,8000,8,3,1,0,74,1,2810,12,...,2,2,2,2,2,1,2,0,0,0
558209,3.0,5000,3,3,0,0,84,1,185,12,...,2,2,2,2,1,1,2,0,0,0


In [ ]:
df=df.drop(['Hperiod'],axis=1)

In [ ]:
Y=df['PotentialFraud']

X=df.iloc[:,:-1]

In [ ]:
X.isna().sum()

claim                         0
period                        0
phy_same                      0
Gender                        0
RenalDiseaseIndicator         0
age                           0
alife                         0
Provider                      0
NoOfMonths_PartACov           0
NoOfMonths_PartBCov           0
ChronicCond_Alzheimer         0
ChronicCond_KidneyDisease     0
ChronicCond_Cancer            0
ChronicCond_ObstrPulmonary    0
ChronicCond_Depression        0
ChronicCond_Diabetes          0
ChronicCond_IschemicHeart     0
ChronicCond_stroke            0
OPAnnualReimbursementAmt      0
OPAnnualDeductibleAmt         0
dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X,Y = oversample.fit_resample(X, Y)

In [ ]:
X

,claim,period,phy_same,Gender,RenalDiseaseIndicator,age,alife,Provider,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_stroke,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,30,0,3,0,0.0,75,1,4004,12,12,2,2,2,2,2,2,2,2,30,50
1,40,0,3,1,0.0,89,1,4004,12,12,1,2,2,2,2,1,1,2,1810,760
2,200,0,3,1,0.0,89,1,4004,12,12,1,2,2,2,2,1,1,2,1810,760
3,20,0,3,1,0.0,89,1,4004,12,12,1,2,2,2,2,1,1,2,1810,760
4,40,0,3,1,0.0,89,1,4004,12,12,1,2,2,2,2,1,1,2,1810,760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690825,80,0,3,0,0.0,81,1,368,12,12,2,2,2,2,1,1,2,2,410,222
690826,200,0,3,0,0.779271,81,1,3197,12,12,1,1,2,2,2,1,1,2,515,797
690827,39,0,3,0,0.0335,82,0,4927,12,12,1,2,2,1,1,1,1,2,220,60
690828,200,0,3,0,0.0,63,1,4954,12,12,1,1,2,1,1,1,1,2,609,90


In [ ]:
X['p']=X['Provider']

In [ ]:
Y.sum()

345415

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y, test_size=2/10, train_size=8/10) 

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression().fit(Xtrain, Ytrain)

In [ ]:
yhat=classifier.predict(Xtest)

In [ ]:
from sklearn.metrics import accuracy_score  
print(f'Accuracy Score is {accuracy_score(Ytest,yhat)}')  

Accuracy Score is 0.5351316532287249


In [ ]:
Xtrain

,claim,period,phy_same,Gender,RenalDiseaseIndicator,age,alife,Provider,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_stroke,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
39152,20,0,3,0,0.0,82,1,1008,12,12,1,2,2,2,2,1,2,2,1110,150
619031,1792,20,3,1,1.0,77,1,4353,12,12,1,1,2,2,2,1,1,2,34450,6780
339608,940,0,3,0,0.0,68,1,1522,12,12,2,2,2,2,1,1,1,2,5720,330
236681,400,0,3,0,0.0,73,1,3018,12,12,1,2,2,2,1,2,2,2,600,180
464560,500,0,3,1,0.0,85,1,2532,12,12,1,2,1,2,2,1,1,2,500,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260975,2000,20,3,0,1.0,94,1,221,12,12,2,1,2,1,2,1,2,2,25540,8200
82120,2600,20,3,1,1.0,68,1,2691,12,12,2,1,1,2,2,1,1,2,13460,3470
498762,50,0,3,0,0.0,74,1,5214,12,12,2,2,2,1,1,2,1,2,610,490
633651,0,0,3,0,0.0,74,1,3026,12,12,1,2,2,2,2,1,1,2,5,200


In [ ]:
model = Sequential(
    [               
        tf.keras.Input(shape=(21,)),    #specify input size
        ### START CODE HERE ### 
        Dense(units=256,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
        Dense(units=128,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
        Dense(units=1,activation='sigmoid',kernel_regularizer=keras.regularizers.l2(0.001))
        
        
        ### END CODE HERE ### 
    ], name = "my_model" 
)     

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 256)               5632      
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
Xtrain = tf.constant(np.asarray(Xtrain), dtype=tf.float32)
Ytrain = tf.constant(np.asarray(Ytrain), dtype=tf.float32)

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.0001),
)

model.fit(Xtrain,Ytrain,epochs=30)

Epoch 1/30
17271/17271 [==============================] - 56s 3ms/step - loss: 0.8234
Epoch 2/30
17271/17271 [==============================] - 55s 3ms/step - loss: 0.7783
Epoch 3/30
17271/17271 [==============================] - 55s 3ms/step - loss: 0.7488
Epoch 4/30
17271/17271 [==============================] - 54s 3ms/step - loss: 0.7316
Epoch 5/30
17271/17271 [==============================] - 55s 3ms/step - loss: 0.7114
Epoch 6/30
17271/17271 [==============================] - 54s 3ms/step - loss: 0.6939
Epoch 7/30
17271/17271 [==============================] - 55s 3ms/step - loss: 0.6848
Epoch 8/30
17271/17271 [==============================] - 56s 3ms/step - loss: 0.6788
Epoch 9/30
17271/17271 [==============================] - 55s 3ms/step - loss: 0.6752
Epoch 10/30
17271/17271 [==============================] - 55s 3ms/step - loss: 0.6737
Epoch 11/30
17271/17271 [==============================] - 54s 3ms/step - loss: 0.6730
Epoch 12/30
17271/17271 [===========================

In [ ]:
Xtest = tf.constant(np.asarray(Xtest), dtype=tf.float32)
Ytest = tf.constant(np.asarray(Ytest), dtype=tf.float32)

In [ ]:
prediction = model.predict(Xtrain)

17271/17271 [==============================] - 39s 2ms/step


In [ ]:
yhut=[]
for i in range(len(prediction)):
    if prediction[i] >= 0.55:
      
        yhut.append(1)
    else:                                #if threshold is high then there will be less False Positive - Prediction a good one as fraud will be more
                                        # if the threshold is low then there will be False Negative -- Prediction a good one as fraud will be less
        yhut.append(0)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
print('Precision: %.3f' % precision_score(Ytrain, yhut))

Precision: 0.682


In [ ]:
print(f'Accuracy Score is {accuracy_score(Ytrain,yhut)}') 

Accuracy Score is 0.5711643964506463


In [ ]:
print('recall: %.3f' % recall_score(Ytrain, yhut))

recall: 0.266


In [ ]:
print('f1 score: %.3f' % f1_score(Ytrain, yhut))

f1 score: 0.383


In [ ]:
prediction


array([[0.7800412 ],
       [0.7748007 ],
       [0.48451248],
       ...,
       [0.37771338],
       [0.47538027],
       [0.23117632]], dtype=float32)